<a href="https://colab.research.google.com/github/peppertaco/AI/blob/main/TavernOAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##태번 실행기

In [ ]:
#@title Tap this if you play on Mobile { display-mode: "form" }
#Taken from KoboldAI colab
%%html
<b>Press play on the music player to keep the tab alive, then start TavernAI below (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title TavernAI
#@markdown **<font color = red><--눌러 (≖ ‸ ≖ ✿)**
#@markdown ###### ***옵션. NGROK 사용 ( 입력시 자동실행 )***
#@markdown - loca.lt 링크보다 ngrok 로딩이 더 빠름 
#@markdown - https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_token = "" #@param {type:"string"}
Model = "GPT4 scale" #@param ["GPT4 scale","GPT4 scale swipe","1.3.1 (public)", "v0.2 (GPT3.5 only)", "Silly Lossy", "Silly Waifu v0.16"] {allow-input: false}
use_google_drive = True #@param {type:"boolean"}
#@markdown - <font color = red> GPT3.5 버그 있습니다. 무한 리퀘스트 발동으로 토큰 쭉쭉 빠져나갑니다.</font>
#@markdown - <font color = red> GPT3.5 를 사용하려면 모든 모델에서 OpenAI API를 선택하면 됩니다.</font>
#@markdown ---
#@markdown ---
#@markdown ---
#@markdown - <font color = red> Silly Lossy 전용 옵션</font>
extras_enable_captioning = True #@param {type:"boolean"}
Captions_Model = "Salesforce/blip-image-captioning-large" #@param [ "Salesforce/blip-image-captioning-large", "Salesforce/blip-image-captioning-base" ]
#@markdown * Salesforce/blip-image-captioning-large - good base model
#@markdown * Salesforce/blip-image-captioning-base - slightly faster but less accurate
extras_enable_emotions = True #@param {type:"boolean"}
Emotions_Model = "joeddav/distilbert-base-uncased-go-emotions-student" #@param ["bhadresh-savani/distilbert-base-uncased-emotion", "joeddav/distilbert-base-uncased-go-emotions-student"]
#@markdown * bhadresh-savani/distilbert-base-uncased-emotion = 6 supported emotions<br>
#@markdown * joeddav/distilbert-base-uncased-go-emotions-student = 28 supported emotions
extras_enable_memory = True #@param {type:"boolean"}
Memory_Model = "Qiliang/bart-large-cnn-samsum-ChatGPT_v3" #@param [ "Qiliang/bart-large-cnn-samsum-ChatGPT_v3", "Qiliang/bart-large-cnn-samsum-ElectrifAi_v10", "distilbart-xsum-12-3" ]
#@markdown * Qiliang/bart-large-cnn-samsum-ChatGPT_v3 - summarization model optimized for chats
#@markdown * Qiliang/bart-large-cnn-samsum-ElectrifAi_v10 - nice results so far, but still being evaluated
#@markdown * distilbart-xsum-12-3 - faster, but pretty basic alternative

!nvidia-smi
!npm install -g localtunnel
import subprocess
import time
import sys
import os
import threading
import shutil
from google.colab import drive

if NGROK_token:
  !pip install pyngrok
  from pyngrok import ngrok
  os.environ['NGROK_token'] = NGROK_token
  !ngrok authtoken $NGROK_token
  !streamlit run app.py&>/dev/null&
  publ_url = ngrok.connect(addr = '8000')
else:
  publ_url = ''
  print("NGROK 미사용")
    
    
if use_google_drive:
  drive.mount('/content/drive/')
  if not os.path.exists("/content/drive/MyDrive/TavernAI/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/characters/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/characters/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/chats/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/chats/")
else:
  if not os.path.exists("/content/drive"):
    os.mkdir("/content/drive")
  if not os.path.exists("/content/drive/MyDrive/"):
    os.mkdir("/content/drive/MyDrive/")

def copy_characters(use_google_drive=False):
  if not use_google_drive:
    return
  
  src_folder = "/TavernAIColab/public/characters"
  dst_folder = "/content/drive/MyDrive/TavernAI/characters"

  for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)

    if os.path.exists(dst_file):
      print(f"{dst_file} already exists. Skipping...")
      continue

    shutil.copy(src_file, dst_folder)
    print(f"{src_file} copied to {dst_folder}")

extras_url = ''
if Model in ["Silly Lossy", "Silly Waifu v0.167"]:
  params = []

  params.append('--cpu')

  ExtrasModules = []

  if (extras_enable_captioning):
   ExtrasModules.append('caption')

  if (extras_enable_memory):
    ExtrasModules.append('summarize')

  if (extras_enable_emotions):
    ExtrasModules.append('classify')

  params.append(f'--classification-model={Emotions_Model}')
  params.append(f'--summarization-model={Memory_Model}')
  params.append(f'--captioning-model={Captions_Model}')
  params.append(f'--enable-modules={",".join(ExtrasModules)}')

#TavernAI extras
  %cd /
  !git clone https://github.com/Cohee1207/TavernAI-extras
  %cd TavernAI-extras
  !pip install -r requirements.txt
  !pip install tensorflow==2.11
  cmd = f"python server.py {' '.join(params)}"
  print(cmd)

  extras_process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd='/TavernAI-extras', shell=True)
  print('processId:', extras_process.pid)

  while True:
      line = extras_process.stdout.readline().decode().strip()
      if "Running on " in line:
         break
      if not line:
         print('breaking on line')
         break
      print(line)

  subprocess.call('nohup lt --port 5100 > ./extras.out 2> ./extras.err &', shell=True)
  print('Waiting for lt init...')
  time.sleep(5)

  while True:
    if (os.path.getsize('./extras.out') > 0):
     with open('./extras.out', 'r') as f:
       lines = f.readlines()
       for x in range(len(lines)):
         if ('your url is: ' in lines[x]):
           print('TavernAI Extensions URL:')
           extras_url = lines[x].split('your url is: ')[1]
           print(extras_url)
       break
    if (os.path.getsize('./extras.err') > 0):
     with open('./extras.err', 'r') as f:
       print(f.readlines())
       break


#TavernAI
%cd /
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
!nvm install 19.1.0
!nvm use 19.1.0
!node -v

if Model == "1.3.1 (public)":
  !git clone https://github.com/peppertaco/TavernAI TavernAIColab
elif Model == "v0.2 (GPT3.5 only)":
  !git clone https://github.com/peppertaco/TavernAIColab
elif Model == "GPT4 scale":
  !git clone -b new4 https://github.com/peppertaco/TavernAIColab  
elif Model == "GPT4 scale swipe":
  !git clone -b new4-swipe https://github.com/peppertaco/TavernAIColab  
elif Model == "Silly Lossy":
  !git clone https://github.com/peppertaco/Tavern TavernAIColab
elif Model == "Silly Waifu v0.16":
  !git clone -b waifu-colab https://github.com/peppertaco/Tavern TavernAIColab 

copy_characters(use_google_drive)
%cd TavernAIColab
!npm install
time.sleep(1)
%env colab=2
%env colaburl=$url
if use_google_drive:
  %env googledrive=2
!nohup node server.js &
time.sleep(3)
print('### TavernAI Extensions LINK ###')
print(extras_url)
print('')
print('###TavernAI LINK###')
print(publ_url)
!lt --port 8000